In [661]:
import numpy as np
import itertools
from scipy.sparse import csr_matrix
import pickle as pk
import sys
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from scripts.data_prepration_scripts.process_concepts import getCleanText
from scipy.sparse.linalg import svds
#import builtin

In [517]:
def create_skill_base(skill_dict):
    
    skill_dict_copy = skill_dict.copy()
    
    for n,i in enumerate(skill_dict):
        skill_dict_copy[i] = sorted(list(skill_dict[i]))
        for j in range(len(skill_dict_copy[i])):
            skill_dict_copy[i][j] = getCleanText(skill_dict_copy[i][j])
        if n%1000==0:
            print(n)
        
    print("Copy Made!")
    
    skill_base = set()
    for n,i in enumerate(skill_dict_copy):
        skill_base = skill_base | set(skill_dict_copy[i])
        if n%1000==0:
            print(n)
        
    skill_base = (list(skill_base))
    print("Cleaning up!")
    for n,i in enumerate(skill_dict_copy):
        skill_dict_copy[i] = list(set(skill_dict_copy[i]))
        if n%1000==0:
            print(n)
    
    return skill_base,skill_dict_copy
    

In [349]:
def generate_word_index_mapping(skills):
    # create word - index mappings
    mappings = {}
    inv_mappings = {}
    for i in range(len(skills)):
        mappings[skills[i]] = i
        inv_mappings[i] = skills[i]
    return mappings, inv_mappings

In [533]:
def create_co_matrix(skill_base, skill_dict):
    count = []
    rows = []
    columns = []
    
    num_skills = len(skill_base)
    
    mappings, inv_map = generate_word_index_mapping(skill_base)
    for n,i in enumerate(skill_dict):
        profile = skill_dict[i]
        if(len(profile)<2):
            continue
        for pair in itertools.combinations(profile,2):
            idx1 = mappings[pair[0]]
            idx2 = mappings[pair[1]]
            rows.append(idx1)
            columns.append(idx2)
            count.append(1)
        if n%3000==0:
            print(n)
    co_occurrence = csr_matrix((count,(rows,columns)), shape = (num_skills,num_skills))
    
    
    
    return mappings,inv_map,co_occurrence

In [263]:
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\linkedin_person_skills_all.pkl",'rb') as f:
    skill_dict = pk.load(f)

In [523]:
%%time
skill_base,skill_dict = create_skill_base(skill_dict)

C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Swapnil\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


0


C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'music'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'videos'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


1000
2000


C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'cookies'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


...

11000
12000


C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'onedrive'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


13000
14000
15000
16000
17000


C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'desktop'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


18000
19000
Copy Made!
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
Cleaning up!
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
Wall time: 31min


In [524]:
#print(len(skill_base))

with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\skill_base_pickle.pkl",'wb') as file:
    pk.dump(skill_base,file)

In [525]:
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\skill_dict_pickle.pkl",'wb') as file:
    pk.dump(skill_dict,file)

In [531]:
print(type(skill_base))
#skill_base.index('')
skill_base[0]
print(type(skill_dict))
#skill_dict.index('')
print(skill_dict[0])
print(skill_2_index_mapping[skill_dict[0][3]])
print(skill_2_index_mapping[skill_dict[0][4]])

<class 'list'>
<class 'dict'>
['computer science', 'linux', 'distributed systems', 'git', 'theory computation', 'distributed databases', 'software engineering', 'haskell', 'artificial intelligence', 'learn skill', 'hadoop', 'mapreduce', 'scalability', 'latex', 'algorithms', 'cassandra', 'python', 'scala', 'java', 'ruby', 'machine learning', 'parallel computing', 'mysql', 'algorithm design']
16592
3885


In [534]:
skill_2_index_mapping, index_2_skill_mapping, skill_co_occurence_matrix = create_co_matrix(skill_base,skill_dict)

0
3000
6000
9000
12000
15000
18000


In [540]:
skill_co_occurence_matrix = skill_co_occurence_matrix.T + skill_co_occurence_matrix

In [731]:
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\skill_2_index_mapping_pickle.pkl",'wb') as file:
    pk.dump(skill_2_index_mapping,file,protocol=2)

In [732]:
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\index_2_skill_mapping_pickle.pkl",'wb') as file:
    pk.dump(index_2_skill_mapping,file,protocol=2)

In [733]:
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\skill_co_occurence_pickle.pkl",'wb') as file:
    pk.dump(skill_co_occurence_matrix,file,protocol=2)

In [536]:
#print((skill_2_index_mapping['machine learning']))
#print((index_2_skill_mapping[17937]))
#print((skill_co_occurence_matrix[24,24])) #shape 18950
#np.sum(skill_co_occurence_matrix[565])
diags = []
for i in range(skill_co_occurence_matrix.shape[0]):
    diags.append(skill_co_occurence_matrix[i,i])

for n,i in enumerate(diags):
    if i!=0:
        print(i,n)


In [610]:
skill_dict[0]
print(skill_2_index_mapping['computer science'])
print(skill_2_index_mapping['finance'])

14159
2771


In [656]:
#print(skill_co_occurence_matrix[14159,0])
#print(skill_co_occurence_matrix[16226,14159])
np.argsort((skill_co_occurence_matrix[skill_2_index_mapping['finance']]))#[::-1][:2]
#skill_co_occurence_matrix = skill_co_occurence_matrix.tocsr()
x = skill_co_occurence_matrix[skill_2_index_mapping['poetry']]
x.todense()
type(x)
x = x.toarray()
y = (x[0].argsort()[::-1][:6])
for i in y:
    print(index_2_skill_mapping[i])

web development
web design
jquery
javascript
creative writing
blogging


In [722]:
#skill_co_occurence_matrix.shape
#testSVD = csr_matrix(skill_co_occurence_matrix, dtype = float)
#testSVD.shape

(18950, 18950)

In [727]:
U,s,Vt = svds(testSVD,k=300)

In [730]:
print(U.shape)
print(s.shape)
print(Vt.shape)
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\SVD_U.pkl",'wb') as file:
    pk.dump(U,file,protocol=2)
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\SVD_s.pkl",'wb') as file:
    pk.dump(s,file,protocol=2)
with open("C:\\Users\\Swapnil\\Downloads\\05899\\Project\\JobSkillRecommendation\\skills\\SVD_Vt.pkl",'wb') as file:
    pk.dump(Vt,file,protocol=2)

(18950, 300)
(300,)
(300, 18950)


In [518]:
#Test
mini_skill_dict = {}
mini_skill_dict[0] = {'','WebD','html 5','SQL','Json','Java','HTML'}#{'HTML','Java','Json','SQL','WebD'}
mini_skill_dict[1] = {'','36','SqL','Java'}
mini_skill_dict[2] = {'JSON','Java','HtML'}
mini_skill_dict[5] = {'HTML','Java'}
mini_skill_dict[7] = {'HTML5'}


In [519]:
#TEst
print(mini_skill_dict)

{0: {'', 'WebD', 'html 5', 'Json', 'HTML', 'Java', 'SQL'}, 1: {'', '36', 'Java', 'SqL'}, 2: {'HtML', 'Java', 'JSON'}, 5: {'HTML', 'Java'}, 7: {'HTML5'}}


In [520]:
#test
list_skill,mini_skill_dict = create_skill_base(mini_skill_dict)
print(list_skill)


0
Copy Made!
0
Cleaning up!
0
['', 'json', 'webd', 'html', 'java', 'html5', 'sql']


C:\Users\Swapnil\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Swapnil\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [675]:
#test
mappings, inv_map,skill_csr = create_co_matrix(list_skill,mini_skill_dict)

0


In [711]:
skill_csrt = csr_matrix((skill_csr+skill_csr.T),dtype = float)
print(skill_csrt.todense())
u,s,vt = svds(skill_csrt,k=3)

[[ 0.  1.  2.  2.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.]
 [ 2.  1.  0.  3.  0.  1.]
 [ 2.  1.  3.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  2.  0.  0.]]


In [714]:

print(u)
#print(s.shape)
print(vt)
#u * np.diag(s) * v
np.dot(u,np.dot(np.diag(s),vt))

[[  8.01505000e-01  -4.51232794e-03   4.46270935e-01]
 [ -1.54043996e-01   5.10610644e-02   3.00065374e-01]
 [ -4.56605650e-01  -6.13245713e-01   5.12998359e-01]
 [ -2.83569310e-01   7.36012227e-01   5.53447534e-01]
 [ -5.55111512e-17  -1.94289029e-16   0.00000000e+00]
 [  2.12037517e-01  -2.82113635e-01   3.75940866e-01]]
[[-0.801505    0.154044    0.45660565  0.28356931  0.         -0.21203752]
 [ 0.00451233 -0.05106106  0.61324571 -0.73601223  0.          0.28211363]
 [ 0.44627094  0.30006537  0.51299836  0.55344753  0.          0.37594087]]


array([[  1.13438442e-01,   1.06269789e+00,   2.08153597e+00,
          1.96857210e+00,   0.00000000e+00,   7.50301503e-01],
       [  1.06269789e+00,   5.16243308e-01,   9.44543231e-01,
          8.47151736e-01,   0.00000000e+00,   8.14214772e-01],
       [  2.08153597e+00,   9.44543231e-01,   7.95873905e-02,
          3.00568886e+00,   0.00000000e+00,   8.30496842e-01],
       [  1.96857210e+00,   8.47151736e-01,   3.00568886e+00,
          4.68188631e-02,   0.00000000e+00,   2.08261849e+00],
       [  7.61432472e-17,   1.66611454e-17,  -4.27333373e-16,
          4.30962150e-16,   0.00000000e+00,  -1.55007280e-16],
       [  7.50301503e-01,   8.14214772e-01,   8.30496842e-01,
          2.08261849e+00,   0.00000000e+00,   5.54371062e-01]])

In [148]:
print(len(skill_dict))
print(skill_dict[0])
print((skill_dict[1]))
print(list((skill_dict[0])|(skill_dict[1])))
#for i in rangeskill_dict:
    

#skill_base = create_skill_base(skill_dict)
#co_matrix = create_co_matrix(skill_base,skill_dict)

19990
{'Machine Learning', 'Python', 'Artificial Intelligence', 'Parallel Computing', 'Distributed Databases', 'Scala', 'LaTeX', 'C', 'Learn more about this skill', 'Hadoop', 'Linux', 'Distributed Systems', 'MapReduce', 'Algorithms', 'Scalability', 'Cassandra', 'MySQL', 'Theory of Computation', 'Computer Science', 'Java', 'Haskell', 'Software Engineering', 'Git', 'Algorithm Design', 'C++', 'Ruby'}
{'Backbone.js', 'Start-ups', 'Web Applications', 'Web Development', 'SQL', 'Cloud Computing', 'GoLang', 'MongoDB', 'Learn more about this skill', 'SaaS', 'Software Development', 'User Experience', 'Management', 'Cassandra', 'JavaScript', 'MySQL', 'User Interface Design', 'Mobile Applications', 'Application Development', 'Photoshop', 'Team Building', 'Git', 'ColdFusion', 'Amazon Web Services (AWS)', 'CSS', 'Business Development'}
['Machine Learning', 'Backbone.js', 'Python', 'Artificial Intelligence', 'Start-ups', 'Web Applications', 'Web Development', 'SQL', 'Cloud Computing', 'Parallel Compu